In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
import pandas as pd
import numpy as np
import re

1. Use a livraria ‘mlxtend’ para extrair os padrões frequentes do dataset, aplicando o algoritmo FPGrowth. Considere um suporte mínimo de 60%. 

In [ ]:
dataset = [['Leite', 'Cerveja', 'Pão'], 
 ['Leite', 'Cerveja', 'Frutas'], 
 ['Leite', 'Cerveja'], 
 ['Leite', 'Pão']]

In [ ]:
te = TransactionEncoder()

df = pd.DataFrame(te.fit(dataset).transform(dataset), columns=te.columns_)
df

In [ ]:
fpgrowth(df, min_support=0.6)

2. Analise o dataset ‘ComprasOnline.xlsx’. 
a) Detete problemas nos dados, como por exemplo existência de NAs e faça um pré 
processamento de limpeza nos mesmos.

In [ ]:
df = pd.read_excel("ComprasOnline.xlsx")
df

In [ ]:
df.info()

In [ ]:
# Remover faturas (invoice) com letras A e C
df_no_alpha_invoice = df[df["InvoiceNo"].apply(lambda x: str(x).isnumeric())]
df_no_alpha_invoice.info()

In [ ]:
# Converter stockcode para string e depois remover todas as letras
df_stockcode = df_no_alpha_invoice.copy()
df_stockcode["StockCode"] = df_stockcode["StockCode"].apply(lambda x: re.sub(r"[a-zA-Z_]+", "", str(x)))

# Remover linhas com stockcode em branco
df_stockcode = df_stockcode[df_stockcode["StockCode"].apply(lambda x: re.match(r"\s*$", str(x)) == None)]

In [ ]:
# Fazer a pivot table
df_france = df_stockcode[df_stockcode["Country"] == "France"]
df_france_reduced = pd.pivot_table(df_stockcode, values = "Quantity", index = ["InvoiceNo", "StockCode"], aggfunc = "sum")

In [ ]:
# Converter quantidades positivas para 1 e negativas para 0
df_france_reduced["Quantity"] = df_france_reduced["Quantity"].apply(lambda x: 1 if x >= 1 else 0)

# Voltar a colocar o dataframe normal
df_final = df_france_reduced.copy()
df_final.reset_index(inplace=True, level = ['InvoiceNo', 'StockCode'])
df_final.info()

Agora é necessário converter para True/False para a biblioteca conseguir execuar o FP-Growth

In [ ]:
dataset = df_final.values.astype(int).tolist()

In [ ]:
te = TransactionEncoder()

itemsets = pd.DataFrame(te.fit(dataset).transform(dataset), columns=te.columns_)
itemsets

In [ ]:
# obtém os itens frequentes
frequent_itemsets = fpgrowth(itemsets, min_support=0.07, use_colnames=True)

In [ ]:
# Obtém as associações dos itens frequentes usando lift
association_rules(frequent_itemsets, metric="lift", min_threshold=6)

In [ ]:
# Obtém as associações dos itens frequentes usando a confiança
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)